In [1]:
import pandas as pd

In [2]:
df_1=pd.read_csv('Компании из data.csv',sep=';')
df_1

,Name,good_name,short_name,division
0,AKKUYU NÜKLEER ANONİM ŞİRKETİ,АО АККУЮ НУКЛЕАР,NaN,5.0
1,TİTAN2 IC İÇTAŞ İNŞAAT A.Ş.,АО АККУЮ НУКЛЕАР,NaN,5.0
2,«Московский проектный институт» Московский фил...,Атомэнергопроект,NaN,1.0
3,«Опытное конструкторское бюро машиностроения и...,Опытное конструкторское бюро машиностроения им...,ОКБМ Африкантов,3.0
4,АКЦИОНЕРНОЕ ОБЩЕСТВО КОНЦЕРН ТИТАН-2,Концерн Титан-2,NaN,5.0
...,...,...,...,...
470,нижнетагильский металлургический комбинат,notset,NaN,NaN
471,филиал АО Концерн Росэнергоатом Калининская АЭС,Калининская АЭС,NaN,5.0
472,филиал АО Концерн Росэнергоатом Нововоронежска...,Нововоронежская Атомная Станция,NaN,5.0
473,филиал АО Концерн Росэнергоатом Опытно-демонст...,Опытно-демонстрационный инженерный центр по вы...,NaN,13.0


In [3]:
titles = df_1.Name
clean = []
import re
for line in titles:
    chis = re.sub(r'(\<(/?[^>]+)>)', ' ', line)
    chis = re.sub('[^а-яА-Я a-zA-Z]', '', line)
    clean.append(chis)
clean[2]

'Московский проектный институт Московский филиал АО Атомэнергопроект'

In [4]:
clean

['AKKUYU NKLEER ANONM RKET',
 'TTAN IC TA NAAT A',
 'Московский проектный институт Московский филиал АО Атомэнергопроект',
 'Опытное конструкторское бюро машиностроения имени И И Африкантова',
 'АКЦИОНЕРНОЕ ОБЩЕСТВО КОНЦЕРН ТИТАН',
 'АКЦИОНЕРНОЕ ОБЩЕСТВО НАУЧНОИССЛЕДОВАТЕЛЬСКИЙ ИНСТИТУТ КОНСТРУКЦИОННЫХ МАТЕРИАЛОВ НА ОСНОВЕ ГРАФИТА НИИГРАФИТ',
 'АО  Атомэнергопроект',
 'АО  Ведущий НаучноИсследовательский Промышленный Институт промышленной технологии',
 'АО ИНСТИТУТ РЕАКТОРНЫХ МАТЕРИАЛОВ',
 'АО  КОНСИСТОС',
 'АО Атомтехэнерго Московский филиал Центратомтехэнерго',
 'АО Атомэнергопроект',
 'АО Атомэнергопроект СанктПетербургский проектный институт ',
 'АО Высокотехнологичный научноисследовательский институт неорганических материалов имени академика АА Бочвара',
 'АО Государственный специализированный проектный институт',
 'АО Государственный специализированный проектный институт ',
 'АО Гринатом',
 'АО Инжиниринговая компания АЭМтехнологии',
 'АО Концерн Росэнергоатом Нововоронежская Ато

In [6]:
import nltk
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")

def token_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[а-яА-Я]', token) or re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

def token_only(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[а-яА-Я]', token)  or re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

#Создаем словари (массивы) из полученных основ
totalvocab_stem = []
totalvocab_token = []
for i in clean:
    allwords_stemmed = token_and_stem(i)
    #print(allwords_stemmed)
    totalvocab_stem.extend(allwords_stemmed)
    
    allwords_tokenized = token_only(i)
    totalvocab_token.extend(allwords_tokenized)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_df=0.8, max_features=10000,
                                 min_df=0.01,
                                 use_idf=True, tokenizer=token_and_stem, ngram_range=(1,3))
get_ipython().magic('time tfidf_matrix = tfidf.fit_transform(clean)')
print(tfidf_matrix.shape)
features = tfidf.fit_transform(titles).toarray() # Remaps the words in the 1490 articles in the text column of 
                                                  # data frame into features (superset of words) with an importance assigned 
                                                  # based on each words frequency in the document and across documents



/tmp/ipykernel_3167778/833226074.py:6: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('time tfidf_matrix = tfidf.fit_transform(clean)')


CPU times: user 170 ms, sys: 2.49 ms, total: 172 ms
Wall time: 172 ms
(475, 221)


In [9]:
from sklearn import preprocessing
labels = df_1.good_name
le = preprocessing.LabelEncoder()
enc=le.fit(labels)
labels=enc.transform(labels)
labels

array([  5,   5,  21,  93,  54,  67,  21,  29,  46,  52,  18,  21,  21,
        33,  38,  38,  40,  44,  74,  63,  93,  89,   3,   4, 115, 123,
       125, 127,   5,  17,  21,  21,  21,  17,  21,  24,  32,  32, 116,
       116,  29, 116,  33,  33,  33,  38,   6,   6,   7,  43,  42,  42,
        43,  43,  43,  43,   8,   8,   9,  52,  52,  54,  52,  52,  52,
        25, 105,  73,  56,  54,  54,  54,  60,  60,  60,  10,  21,  63,
        93,  93,  93,  93,  36,  11,  12, 112, 119, 119, 115, 130, 130,
       121, 122, 123, 123, 122, 123, 125, 125, 127, 127, 127, 128, 131,
         9,  54,   2,  49, 112,  66,  66,  63,   6,  41,  46,  60, 115,
        59,  16,  29,  54,  54, 102, 107, 106, 115, 123,  14,  15,  16,
        17,  20,  20,  21,  15,  29,  33,  33, 133,  38,   6,   7,   8,
        46,  46,  38,  60,  59,  59,  93,  93, 102, 103, 107, 106, 112,
       112, 114, 115, 121, 122, 122, 125, 127, 127,  93,   2,   2,  17,
         9,  23,  65,  24,  24,  24,  24,  25,  26,  65, 116,   

In [19]:
import numpy as np
len(np.unique(labels))

134

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier


parameters = {
    'learning_rate' : [0.01,0.02,0.03],
    'n_estimators' : [150,200,250],
    'max_depth':[20,30,40],
}

gbc = GradientBoostingClassifier(random_state=12345)
clf_gbc = GridSearchCV(
    estimator = gbc,
    param_grid = parameters,
    scoring = 'f1_macro',
    cv=5)
clf_gbc.fit(features, labels)

/home/jupyter/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [22]:
clf_gbc.best_params_

{'learning_rate': 0.01, 'max_depth': 20, 'n_estimators': 150}

In [ ]:
text_features = tfidf.transform(clean)
predictions = model.predict(text_features)
predictions=le.inverse_transform(predictions)
for text, predicted in zip(clean, predictions):
  print('"{}"'.format(text))
  print("  - Predicted as: '{}'".format(predicted))
